# INFO 4271 - Group Project

Issued: June 11, 2024

Due: July 22, 2024

Please submit a link to your code base (ideally with a branch that does not change anymore after the submission deadline) and your 4-page report via email to carsten.eickhoff@uni-tuebingen.de by the due date. One submission per team.

---

# 1. Web Crawling & Indexing
Crawl the web to discover **English content related to Tübingen**. The crawled content should be stored locally. If interrupted, your crawler should be able to re-start and pick up the crawling process at any time.

In [24]:
import sqlite3
import requests
from bs4 import BeautifulSoup
import datetime
from langdetect import detect, LangDetectException
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from urllib.parse import urljoin, urlparse

DB_NAME = "index.db"
NUM_WORKERS = 10
FILTER_CONTENT = True
TIMEOUT = 15
TUEBINGEN_KEYWORDS = ['tübingen', 'tubingen', 'tuebingen', 'neckar', 'schwaben', 'swabia', 'university', 'uni',
                      't%c3%bcbingen']

def setup_database(db_name=DB_NAME, drop_existing=False):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # option to reset the database
    if drop_existing:
        cursor.execute("DROP TABLE IF EXISTS frontier")
        cursor.execute("DROP TABLE IF EXISTS documents")
        cursor.execute("DROP TABLE IF EXISTS incoming_links")

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS frontier (
        url TEXT PRIMARY KEY,
        crawled INTEGER DEFAULT 0
    )''')
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS documents (
        url TEXT PRIMARY KEY,
        title TEXT,
        content TEXT,
        outgoing_links TEXT,
        timestamp TEXT
    )''')
    conn.commit()
    conn.close()

def index_doc(doc, index_path):
    conn = sqlite3.connect(index_path)
    cursor = conn.cursor()
    cursor.execute('''
    INSERT OR IGNORE INTO documents (url, title, content, outgoing_links, timestamp)
    VALUES (?, ?, ?, ?, ?)
    ''', (doc['url'], doc['title'], doc['content'], ','.join(doc['outgoing_links']), doc['timestamp']))
    conn.commit()

def crawl_page(url):
    try:

        response = requests.get(url, timeout=TIMEOUT)  # Fetch the web page
        if response.status_code != 200:
            print(f"Error while fetching response from {url} (response status {response.status_code})")

            # remove URL from frontier if status code is 404
            if response.status_code == 404:
                return "404"

            return None

        soup = BeautifulSoup(response.text, 'html.parser')  # Parse the HTML content
        title = soup.title.string if soup.title else "N/A"
        content = ' '.join(soup.stripped_strings)  # Using stripped_strings to clean up the text

        # Filter out pages that do not contain "tuebingen" in their content or are not in English
        if FILTER_CONTENT:
            if not any(word in content.lower() for word in ['tübingen', 'tubingen', 'tuebingen']):
                return None

            try:
                if detect(content) != 'en':
                    return None
            except LangDetectException:
                return None

        # get all outgoing links and transform them into absolute URLs
        links = [link.get('href') for link in soup.find_all('a', href=True)]
        absolute_links = []
        for link in links:
            # Create absolute URL and parse it
            abs_url = urljoin(url, link)
            parsed_url = urlparse(abs_url)

            # Check if the URL is valid (i.e., it has a network location part and scheme)
            if parsed_url.scheme and parsed_url.netloc:
                absolute_links.append(abs_url)

        links = list(set(absolute_links))  # Remove duplicates

        doc = {
            'url': url,
            'title': title,
            'content': content,
            'outgoing_links': links,
            'timestamp': datetime.datetime.now().isoformat()
        }

    except requests.RequestException as e:
        print(f"Request exception encountered at {url}: {e}")
        return None
    except Exception as e:
        print(f"Unexpected exception encountered at {url}: {e}")
        return None

    return doc

def crawl(index_path):
    conn = sqlite3.connect(index_path)
    cursor = conn.cursor()

    # Calculate total number of URLs to be crawled
    cursor.execute("SELECT COUNT(*) FROM frontier WHERE crawled = 0")
    total_to_crawl = cursor.fetchone()[0]

    with tqdm(total=total_to_crawl, desc="Crawling Progress", unit="page") as pbar:
        while True:
            cursor.execute("SELECT url FROM frontier WHERE crawled = 0 LIMIT 10")
            rows = cursor.fetchall()
            if not rows:
                break

            urls = [row[0] for row in rows]
            with ThreadPoolExecutor(max_workers=NUM_WORKERS) as executor:
                results = executor.map(lambda url: crawl_page(url), urls)

            for url, doc in zip(urls, results):
                if doc:

                    # Remove URL from frontier if status code is 404
                    if doc == "404":
                        cursor.execute("DELETE FROM frontier WHERE url = ?", (url,))
                        conn.commit()
                        pbar.update(1)
                        continue

                    cursor.execute("SELECT 1 FROM documents WHERE url = ? LIMIT 1", (doc['url'],))
                    if cursor.fetchone() is None:
                        index_doc(doc, index_path)
                        pbar.update(1)

                cursor.execute("UPDATE frontier SET crawled = 1 WHERE url = ?", (url,))
                conn.commit()
                if doc:
                    for link in doc['outgoing_links']:
                        cursor.execute("INSERT OR IGNORE INTO frontier (url) VALUES (?)", (link,))
                        conn.commit()
    
    conn.close()

def initialize_frontier(initial_urls, db_name=DB_NAME):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    for url in initial_urls:
        cursor.execute("INSERT OR IGNORE INTO frontier (url) VALUES (?)", (url,))
    conn.commit()
    conn.close()

    print(f"Inserted {len(initial_urls)} URLs into the frontier table")

    return None

def calculate_incoming_links(db_name=DB_NAME):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # Create a temporary table to store incoming links
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS incoming_links (
        url TEXT PRIMARY KEY,
        incoming_count INTEGER DEFAULT 0
    )''')

    # Clear any existing data
    cursor.execute("DELETE FROM incoming_links")

    # Retrieve all documents and their outgoing links
    cursor.execute("SELECT url, outgoing_links FROM documents")
    rows = cursor.fetchall()

    for row in rows:
        url, outgoing_links = row
        outgoing_links_list = outgoing_links.split(',')

        for link in outgoing_links_list:
            cursor.execute('''
            INSERT INTO incoming_links (url, incoming_count)
            VALUES (?, 1)
            ON CONFLICT(url) DO UPDATE SET incoming_count = incoming_count + 1
            ''', (link,))

    conn.commit()
    conn.close()

# Initialize and run the crawler
initial_urls = [
    "https://www.tuebingen.de/en/",
    "https://en.wikipedia.org/wiki/T%C3%BCbingen",
    "https://www.uni-tuebingen.de/en.html"
]

setup_database()
initialize_frontier(initial_urls)
crawl("index.db")
calculate_incoming_links()

Inserted 3 URLs into the frontier table


Crawling Progress:   0%|          | 0/126497 [00:00<?, ?page/s]

Error while fetching response from https://en.wikipedia.org/w/index.php?title=J_Hermann_Siemer&action=edit&redlink=1 (response status 404)
Error while fetching response from https://en.wikipedia.org/w/index.php?title=Max_Schulze-Vorberg&action=edit&redlink=1 (response status 404)
Error while fetching response from https://en.wikipedia.org/w/index.php?title=Karl_Moersch&action=edit&redlink=1 (response status 404)


Crawling Progress:   0%|          | 1/126497 [00:01<48:10:23,  1.37s/page]

Error while fetching response from https://en.wikipedia.org/w/index.php?title=Berthold_Martin&action=edit&redlink=1 (response status 404)
Error while fetching response from https://en.wikipedia.org/w/index.php?title=Dionys_Jobst&action=edit&redlink=1 (response status 404)


...

Crawling Progress:   0%|          | 509/126497 [04:08<15:58:33,  2.19page/s]

Error while fetching response from https://en.wikipedia.org/w/index.php?title=Heinz_Eyrich&action=edit&redlink=1 (response status 404)


Crawling Progress:   0%|          | 517/126497 [04:11<14:10:32,  2.47page/s]

Error while fetching response from https://en.wikipedia.org/w/index.php?title=Peter_S%C3%A4ckl&action=edit&redlink=1 (response status 404)


Crawling Progress:   0%|          | 553/126497 [04:51<25:57:20,  1.35page/s]

In [19]:
def print_first_10_urls(db_name=DB_NAME):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    cursor.execute("SELECT title, url FROM documents LIMIT 20")
    rows = cursor.fetchall()
    for row in rows:
        print(row[0], row[1])

    conn.close()

# Call the function to print the first 10 URLs
print_first_10_urls()

Welcome to Tübingen - City of Tuebingen https://www.tuebingen.de/en/
Tübingen - Wikipedia https://en.wikipedia.org/wiki/T%C3%BCbingen
Culture and Leisure - City of Tuebingen https://www.tuebingen.de/en/3515.html
Welcome to Tübingen - City of Tuebingen https://www.tuebingen.de/en
Portrait of the City - City of Tuebingen https://www.tuebingen.de/en/1815.html
Imprint - City of Tuebingen https://www.tuebingen.de/en/1821.html
Immigration office - City of Tuebingen https://www.tuebingen.de/en/42067.html
European Elections 2024 - City of Tuebingen https://www.tuebingen.de/en/42070.html
City and Guests - City of Tuebingen https://www.tuebingen.de/en/3494.html
Ralf Dahrendorf - Wikipedia https://en.wikipedia.org/wiki/Ralf_Dahrendorf
Tübingen - Wikipedia https://en.wikipedia.org/wiki/T%C3%BCbingen#cite_ref-14
Johann Friedrich Cotta - Wikipedia https://en.wikipedia.org/wiki/Johann_Friedrich_Cotta
Pfrondorf - Wikipedia https://en.wikipedia.org/wiki/Pfrondorf
Tübingen - Wikipedia https://en.wikiped

In [21]:
def get_total_indexed_docs(db_name=DB_NAME):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    cursor.execute("SELECT count(*) FROM documents")
    total_count = cursor.fetchone()
    print(f"Total number of documents indexed: {total_count[0]}")

    conn.close()

get_total_indexed_docs()

Total number of documents indexed: 489


In [22]:
def count_remaining_frontier(db_name=DB_NAME):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    cursor.execute("SELECT count(*) FROM frontier WHERE crawled = 0")
    total_count = cursor.fetchone()
    print(f"Total number of URLs remaining in the frontier: {total_count[0]}")

    conn.close()

count_remaining_frontier()

Total number of URLs remaining in the frontier: 126406


In [5]:
#Add a document to the index. You need (at least) two parameters:
	#doc: The document to be indexed.
	#index: The location of the local index storing the discovered documents.
def index_doc(doc, index_path):
    conn = sqlite3.connect(index_path)
    cursor = conn.cursor()
    cursor.execute('''
    INSERT OR IGNORE INTO documents (url, title, content, outgoing_links, timestamp)
    VALUES (?, ?, ?, ?, ?)
    ''', (doc['url'], doc['title'], doc['content'], ','.join(doc['outgoing_links']), doc['timestamp']))
    conn.commit()
    
import spacy

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

def is_relevant(content):
    # Check for simple keyword presence
    if not any(word in content for word in ['Tübingen', 'Tubingen', 'Tuebingen', 'tübingen', 'tubingen', 'tuebingen']):
        return False
    
    # Perform NER and check for geographic relevance
    doc = nlp(content)
    for ent in doc.ents:
        if ent.label_ == "GPE" and "tübingen" in ent.text.lower():
            return True
    return False

def index_doc(doc, index_path):
    if not is_relevant(doc['content']):
        #print("Skipping non-relevant document.")
        return

    conn = sqlite3.connect(index_path)
    cursor = conn.cursor()
    cursor.execute('''
    INSERT OR IGNORE INTO documents (url, title, content, outgoing_links, timestamp)
    VALUES (?, ?, ?, ?, ?)
    ''', (doc['url'], doc['title'], doc['content'], ','.join(doc['outgoing_links']), doc['timestamp']))
    conn.commit()
    conn.close()

    conn.close()

/Users/Felix/opt/anaconda3/envs/MSE_project/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as Service
from selenium.webdriver.chrome.options import Options
#from webdriver_manager.chrome import ChromeDriverManager
import time
from bs4 import BeautifulSoup
import datetime
from langdetect import detect, LangDetectException
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

def crawl_page(url, index_path):
    options = Options()
    options.headless = True
    options.add_argument('--no-sandbox')
    service = Service("/opt/homebrew/Caskroom/chromedriver/126.0.6478.126/chromedriver-mac-arm64/chromedriver")
    driver = webdriver.Chrome(service=service, options=options)
    try:
        driver.get(url)
        time.sleep(3)  # Wait for the page to load

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        title = soup.title.string if soup.title else ""
        content = soup.get_text(separator=' ', strip=True)

        # Check if the page contains the word "Tübingen"
        if not any(word in content for word in ['Tübingen', 'Tubingen', 'Tuebingen', 'tübingen', 'tubingen', 'tuebingen']):
            #print(f"Skipping url {url} Page does not contain Tübingen...")
            return None

        # Check if English content is present
        if not any(word in content for word in ['the', 'and', 'is', 'in']):
            #print(f"Skipping url {url} Page does not contain English content")
            return None

        # Language detection
        try:
            if detect(content) != 'en':
                #print(f"Skipping url {url} Page does not contain English content")
                return None
        except LangDetectException:
            return None

        links = [link.get_attribute("href") for link in driver.find_elements(By.TAG_NAME, "a") if link.get_attribute("href") and "tuebingen" in link.get_attribute("href").lower()]
        doc = {
            'url': url,
            'title': title,
            'content': content,
            'outgoing_links': links,
            'timestamp': datetime.datetime.now().isoformat()
        }

    except (StaleElementReferenceException, TimeoutException, NoSuchElementException) as e:
        print(f"Exception {type(e).__name__} encountered at {url}: {e}")
        return None
    except Exception as e:
        print(f"Unexpected exception encountered at {url}: {e}")
        return None
    finally:
        driver.quit()

    return doc

def crawl(frontier, index_path):
    conn = sqlite3.connect(index_path)
    cursor = conn.cursor()

    # Calculate total number of URLs to be crawled
    cursor.execute("SELECT COUNT(*) FROM frontier WHERE crawled = 0")
    total_to_crawl = cursor.fetchone()[0]

    with tqdm(total=total_to_crawl, desc="Crawling Progress", unit="page") as pbar:
        while True:
            cursor.execute("SELECT url FROM frontier WHERE crawled = 0 LIMIT 10")
            rows = cursor.fetchall()
            if not rows:
                break

            urls = [row[0] for row in rows]
            with ThreadPoolExecutor(max_workers=2) as executor:
                results = executor.map(lambda url: crawl_page(url, index_path), urls)

            for url, doc in zip(urls, results):
                if doc:
                    cursor.execute("SELECT 1 FROM documents WHERE url = ? LIMIT 1", (doc['url'],))
                    if cursor.fetchone() is None:
                        index_doc(doc, index_path)
                        pbar.update(1)
                cursor.execute("UPDATE frontier SET crawled = 1 WHERE url = ?", (url,))
                conn.commit()
                if doc:
                    for link in doc['outgoing_links']:
                        cursor.execute("INSERT OR IGNORE INTO frontier (url) VALUES (?)", (link,))
                        conn.commit()
    
    conn.close()

def initialize_frontier(initial_urls, db_name="crawler.db"):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    for url in initial_urls:
        cursor.execute("INSERT OR IGNORE INTO frontier (url) VALUES (?)", (url,))
    conn.commit()
    conn.close()

def calculate_incoming_links(db_name="crawler.db"):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # Create a temporary table to store incoming links
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS incoming_links (
        url TEXT PRIMARY KEY,
        incoming_count INTEGER DEFAULT 0
    )''')

    # Clear any existing data
    cursor.execute("DELETE FROM incoming_links")

    # Retrieve all documents and their outgoing links
    cursor.execute("SELECT url, outgoing_links FROM documents")
    rows = cursor.fetchall()

    for row in rows:
        url, outgoing_links = row
        outgoing_links_list = outgoing_links.split(',')

        for link in outgoing_links_list:
            cursor.execute('''
            INSERT INTO incoming_links (url, incoming_count)
            VALUES (?, 1)
            ON CONFLICT(url) DO UPDATE SET incoming_count = incoming_count + 1
            ''', (link,))

    conn.commit()
    conn.close()

# Initialize and run the crawler
initial_urls = [
    "https://www.tuebingen.de/en/",
    #"https://en.wikipedia.org/wiki/T%C3%BCbingen",
    #"https://www.uni-tuebingen.de/en.html"
]

setup_database()
initialize_frontier(initial_urls)
crawl("crawler.db", "crawler.db")
calculate_incoming_links()

ModuleNotFoundError: No module named 'selenium'

In [7]:
import requests
from bs4 import BeautifulSoup
import datetime
from langdetect import detect, LangDetectException

def crawl_page(url, index_path):
    try:
        response = requests.get(url, timeout=10)  # Fetch the web page
        if response.status_code != 200:
            return None

        soup = BeautifulSoup(response.text, 'html.parser')
        title = soup.title.string if soup.title else ""
        content = ' '.join(soup.stripped_strings)  # Using stripped_strings to clean up the text

        # Check if the page contains the word "Tübingen"
        if not any(word in content.lower() for word in ['tübingen', 'tubingen', 'tuebingen']):
            return None

        # Check if English content is present
        if not any(word in content.lower() for word in ['the', 'and', 'is', 'in']):
            return None

        # Language detection
        try:
            if detect(content) != 'en':
                return None
        except LangDetectException:
            return None

        # Extract outgoing links that contain "tuebingen"
        links = [a['href'] for a in soup.find_all('a', href=True) if "tuebingen" in a['href'].lower()]

        doc = {
            'url': url,
            'title': title,
            'content': content,
            'outgoing_links': links,
            'timestamp': datetime.datetime.now().isoformat()
        }

    except requests.RequestException as e:
        print(f"Request exception encountered at {url}: {e}")
        return None
    except Exception as e:
        print(f"Unexpected exception encountered at {url}: {e}")
        return None

    return doc

ModuleNotFoundError: No module named 'bs4'

In [4]:
def print_first_10_urls(db_name="crawler.db"):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    cursor.execute("SELECT title, url FROM documents LIMIT 20")
    rows = cursor.fetchall()
    for row in rows:
        print(row[0], row[1])

    conn.close()

# Call the function to print the first 10 URLs
print_first_10_urls()

Welcome to Tübingen - City of Tuebingen https://www.tuebingen.de/en/
Tübingen - Wikipedia https://en.wikipedia.org/wiki/T%C3%BCbingen
Welcome to Tübingen - City of Tuebingen https://www.tuebingen.de/en
Immigration office - City of Tuebingen https://www.tuebingen.de/en/42067.html
European Elections 2024 - City of Tuebingen https://www.tuebingen.de/en/42070.html
Portrait of the City - City of Tuebingen https://www.tuebingen.de/en/1815.html
City and Guests - City of Tuebingen https://www.tuebingen.de/en/3494.html
Culture and Leisure - City of Tuebingen https://www.tuebingen.de/en/3515.html
Imprint - City of Tuebingen https://www.tuebingen.de/en/1821.html
Welcome to Tübingen - City of Tuebingen https://www.tuebingen.de/en/#
File:Altstadt-tuebingen-1.jpg - Wikipedia https://en.wikipedia.org/wiki/File:Altstadt-tuebingen-1.jpg
File:Wappen Tuebingen.svg - Wikipedia https://en.wikipedia.org/wiki/File:Wappen_Tuebingen.svg
File:TuebingenNeckar.jpg - Wikipedia https://en.wikipedia.org/wiki/File:Tu

In [4]:
import math
from collections import Counter

def fetch_documents(index_path):
    conn = sqlite3.connect(index_path)
    cursor = conn.cursor()
    cursor.execute("SELECT content FROM documents")
    documents = [doc[0] for doc in cursor.fetchall()]
    conn.close()
    return documents

def bm25(documents, query, k1=1.5, b=0.75):
    # Tokenize documents and query
    tokenized_docs = [doc.lower().split() for doc in documents]
    tokenized_query = query.lower().split()

    # Calculate document frequencies
    df = {}
    for word in tokenized_query:
        df[word] = sum(word in doc for doc in tokenized_docs)

    # Calculate IDF
    N = len(documents)
    idf = {word: math.log((N - df[word] + 0.5) / (df[word] + 0.5)) for word in df}

    # Compute BM25 scores
    scores = []
    avgdl = sum(len(doc) for doc in tokenized_docs) / N
    for index, doc in enumerate(tokenized_docs):
        doc_len = len(doc)
        f = Counter(doc)
        score = sum(idf[word] * f[word] * (k1 + 1) / (f[word] + k1 * (1 - b + b * doc_len / avgdl)) for word in tokenized_query if word in f)
        scores.append((index, score))
    
    scores.sort(key=lambda x: x[1], reverse=True)
    return scores

documents = fetch_documents("crawler.db")
query = "Tuebingen"
results = bm25(documents, query)
print(results)

[(642, 4.775923434800474), (640, 4.772457630479728), (908, 4.767770040902589), (906, 4.764316055753173), (517, 4.730409989359467), (515, 4.726479081441341), (188, 4.054680750861058), (1464, 4.054680750861058), (1465, 4.054680750861058), (1466, 4.054680750861058), (1467, 4.054680750861058), (1468, 4.054680750861058), (1469, 4.054680750861058), (1470, 4.054680750861058), (1471, 4.054680750861058), (1472, 4.054680750861058), (1473, 4.054680750861058), (1474, 4.054680750861058), (1475, 4.054680750861058), (6, 3.9613292950762737), (33, 3.9613292950762737), (764, 3.8941387383641075), (753, 3.806703113051436), (111, 3.744614494818716), (749, 3.744614494818716), (752, 3.744614494818716), (763, 3.744614494818716), (1596, 3.679187373316038), (201, 3.589702882214677), (1044, 3.5636051636334285), (1048, 3.56168557335207), (914, 3.5532638862560453), (923, 3.454863322129475), (150, 3.4530590656906055), (22, 3.418466973778695), (169, 3.418466973778695), (37, 3.405789140766824), (212, 3.40578914076682

In [1]:
import torch
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')

def semantic_search(documents, query):
    doc_embeddings = model.encode(documents, convert_to_tensor=True)
    query_embedding = model.encode(query, convert_to_tensor=True)
    scores = util.pytorch_cos_sim(query_embedding, doc_embeddings)[0]
    ranked_results = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)
    return ranked_results

documents = fetch_documents("crawler.db")
query = "Example search query"
results = semantic_search(documents, query)
print(results)

/Users/Felix/opt/anaconda3/envs/MSE_project/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AttributeError: module 'torch.backends' has no attribute 'mps'

# 2. Query Processing 
Process a textual query and return the 100 most relevant documents from your index. Please incorporate **at least one retrieval model innovation** that goes beyond BM25 or TF-IDF. Please allow for queries to be entered either individually in an interactive user interface (see also #3 below), or via a batch file containing multiple queries at once. The batch file will be formatted to have one query per line, listing the query number, and query text as tab-separated entries. An example of the batch file for the first two queries looks like this:

```
1   tübingen attractions
2   food and drinks
```

In [ ]:
#Retrieve documents relevnt to a query. You need (at least) two parameters:
	#query: The user's search query
	#index: The location of the local index storing the discovered documents.
def retrieve(query, index):
    #TODO: Implement me
	pass

# 3. Search Result Presentation
Once you have a result set, we want to return it to the searcher in two ways: a) in an interactive user interface. For this user interface, please think of **at least one innovation** that goes beyond the traditional 10-blue-links interface that most commercial search engines employ. b) as a text file used for batch performance evaluation. The text file should be formatted to produce one ranked result per line, listing the query number, rank position, document URL and relevance score as tab-separated entries. An example of the first three lines of such a text file looks like this:

```
1   1   https://www.tuebingen.de/en/3521.html   0.725
1   2   https://www.komoot.com/guide/355570/castles-in-tuebingen-district   0.671
1   3   https://www.unimuseum.uni-tuebingen.de/en/museum-at-hohentuebingen-castle   0.529
...
1   100 https://www.tuebingen.de/en/3536.html   0.178
2   1   https://www.tuebingen.de/en/3773.html   0.956
2   2   https://www.tuebingen.de/en/4456.html   0.797
...
```

In [ ]:
#TODO: Implement an interactive user interface for part a of this exercise.

#Produce a text file with 100 results per query in the format specified above.
def batch(results): 
    #TODO: Implement me.    
    pass

# 4. Performance Evaluation 
We will evaluate the performance of our search systems on the basis of five queries. Two of them are avilable to you now for engineering purposes:
- `tübingen attractions`
- `food and drinks`

The remaining three queries will be given to you during our final session on July 23rd. Please be prepared to run your systems and produce a single result file for all five queries live in class. That means you should aim for processing times of no more than ~1 minute per query. We will ask you to send carsten.eickhoff@uni-tuebingen.de that file.

# Grading
Your final projects will be graded along the following criteria:
- 25% Code correctness and quality (to be delivered on this sheet)
- 25% Report (4 pages, PDF, explanation and justification of your design choices)
- 25% System performance (based on how well your system performs on the 5 queries relative to the other teams in terms of nDCG)
- 15% Creativity and innovativeness of your approach (in particular with respect to your search system #2 and user interface #3 innovations)
- 10% Presentation quality and clarity

# Permissible libraries
You can use any general-puprose ML and NLP libraries such as scipy, numpy, scikit-learn, spacy, nltk, but please stay away from dedicated web crawling or search engine toolkits such as scrapy, whoosh, lucene, terrier, galago and the likes. Pretrained models are fine to use as part of your system, as long as they have not been built/trained for retrieval. 
